# Documentation site

> A documentation website that dynamically loads demos and source code from the source notebooks

In [ ]:
#| export
from fasthtml.common import *
from basecampui.all import *
import fasthtml.components as fh
from fasthtml.jupyter import *
from fastcore.meta import delegates
from fastcore.utils import *
import nbformat
import itertools
from functools import wraps
import json

### Initial setup

In [ ]:
custom_highlighting = " ".join(name for name in globals() if name[0].isupper())
app = FastHTML(
    hdrs=(Link(rel="icon", href="./favicon.png"),),
    custom_kws=custom_highlighting, exts=("ws", "preload"), session_cookie="mysession",
    title="BasecampUI",
)
rt = app.route

In [ ]:
@app.middleware("http")
async def add_cache_headers(request, call_next):
    response = await call_next(request)
    if "/documentation-rag/" not in str(request.url):
        response.headers["Cache-Control"] = "max-age=300"  # 5 minutes
    return response

In [ ]:
srv = JupyUvi(app)

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/base.p

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 244, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/site-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/errors.py", line 151, in __call__
    await self.app(scope, receive, send)
  File "/usr/local/lib/python3.12/site-packages/starlette/middleware/base.py", line 103, in __call__
    await self.app(

In [ ]:
slug_map = {}

In [ ]:
def make_key():
    current = [None]
    def key_fn(cell):
        if cell.cell_type == "markdown" and cell.source.startswith("###"):
            current[0] = cell.source.strip("###").strip()
        return current[0]
    return key_fn

In [ ]:
def mk_nbdict(filename:str):
    nb = nbformat.read(filename, as_version=4)
    cells = itertools.islice(itertools.dropwhile(lambda c: c.source != "# Components", nb.cells), 1, None)
    nbdict = {}
    for header, group in itertools.groupby(cells, key=make_key()):
        header_slug = slugify(header)
        slug_map[header_slug] = header
        nbdict[header_slug] = list(group)
    return nbdict 


In [ ]:
mk_nbdict("04_chat.ipynb").keys()

dict_keys(['chat-input', 'chat-message', 'chat-code', 'chat-pairs'])

In [ ]:
matched = re.search(r"\d+_(\w+)\.ipynb", "01_common.ipynb").group(1)
matched

'common'

In [ ]:
def format_nm(nm:str):
    return re.search(r"\d+_(\w+)\.ipynb", nm).group(1).title()

In [ ]:
documentation = {format_nm(nm): mk_nbdict(nm) for nm in ["01_common.ipynb", "02_interactive.ipynb", "03_markdown.ipynb", "04_chat.ipynb"]}

In [ ]:
documentation.keys()

dict_keys(['Common', 'Interactive', 'Markdown', 'Chat'])

### Search bar

In [ ]:
def SearchBarButton(id):
    return DialogOpenButton(
        Div(
            Div(
                Icon("search", cls="size-4"),
                Span("Search Documentation", cls="text-sm"),
                cls="flex gap-2 items-center",
            ),
            Kbd("⌘ K", cls="text-2xs"),
            cls="flex items-center px-3 py-2 justify-between",
        ),
        cls="m-1 hover:cursor-pointer bg-background rounded-lg w-full",
        did=id,
    )

In [ ]:
def mk_command_items(category:str, nms:list[str]):
    return [
        A(
            CommandItem(slug_map[nm]), 
            href=f"/{category.lower()}/{nm}",
            cls="block w-full",
            onclick="this.closest('dialog').close(); document.activeElement.blur()"
        )
        for nm in nms
    ]

In [ ]:
def SearchBarCommand(id):
    groups = [
        CommandGroup(
            ItemHeader(category), 
            *mk_command_items(category, nbdict.keys())
        )
        for category, nbdict in documentation.items()
    ]
    
    return CommandDialog(
        Command(
            CommandSearch(),
            CommandScrollable(*groups, style="height: 400px;"),
            cls="w-4xl",
            style="top: 29%; max-width: 650px; max-height: 500px;",

        ),
        id=id,


    )

In [ ]:
def SearchBar(id="search-bar"):
    return Div(
        SearchBarButton(id=id),
        SearchBarCommand(id=id),
        Script(f"""
        document.addEventListener('keydown', (e) => {{
            if ((e.ctrlKey || e.metaKey) && e.key === 'k') {{
                e.preventDefault();
                const dialog = document.getElementById("{id}");
                if (dialog.open) {{
                    dialog.close();
                }} else {{
                    dialog.showModal();
                    dialog.querySelector("header input")?.focus();
                }}
            }}
        }});
        """),
        cls="flex w-full",
    )

### Sidebar

In [ ]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read('../settings.ini')
ver = cfg['DEFAULT']['version']
ver

'0.0.1'

In [ ]:
def DropdownFooter():
    return Div(
            Dropdown(
                DropdownItem(Icon("heart", cls="size-4"), "Made for FastHTML"),
                Separator(),
                DropdownItem(Icon("tag", cls="size-4"), f"Version {ver}"),
                A(DropdownItem(Icon("bug", cls="size-4"), "Report Bug", cls="cursor-pointer"), 
                  href="https://github.com/taya-nicholas/basecampui/issues/new"),
                trigger_btn=DropdownTriggerButton(
                    Div(
                        Avatar(src="https://ui.shadcn.com/avatars/shadcn.jpg"), 
                        Span("Taya", cls="pl-2"),
                        cls="flex items-center w-full",
                    ),
                    Icon("chevrons-up-down"), did="sidebar-dropdown", 
                    cls=f"{ButtonT.ghost} justify-between py-6 px-3 m-1",
                    style="width: calc(100% - 0.5rem)"
                ),
                id="sidebar-dropdown",
                side="right", align="end",
        ),
        cls="flex flex-col items-left w-full",
    )

In [ ]:
category_icons = {"Common": "blocks", "Interactive": "mouse-pointer-click", "Markdown": "heading-1", "Chat": "message-square"}

In [ ]:
@rt("/{fname:path}.{ext:static}")
async def get(fname: str, ext: str): return FileResponse(f'{fname}.{ext}')

In [ ]:
def TitleButton():
    return A(
        Button(
            Span(Img(src="/favicon.png", cls="size-10"), cls="text-2xl"),
            H3("BASECAMPUI", cls="text-xl font-bold tracking-wider"),
            cls="flex items-center gap-2 px-4 py-2 rounded-lg transition-colors duration-200 !p-5 mt-1 mx-auto" + ButtonT.ghost,
            style="width: calc(100% - 0.5rem)",
        ),
        href="/",
        cls="no-underline"
    )
p(TitleButton())

In [ ]:
def SiteSidebar():
    contents = [
        SidebarCollapsable(
            IconTitle(category, icon=category_icons.get(category)), 
            [slug_map[k] for k in nbdict.keys()],
            href_list=nbdict.keys(),
            href_parent=category,
        ) 
        for category, nbdict in documentation.items()
    ]
    return Sidebar(
        TitleButton(),
        SearchBar(),
        Group(
            "Components", 
            *contents
        ),
        SidebarGroup("Demos", ["Documentation RAG"], icon_list=["bot"], href_parent="demos"),
        SidebarGroup("Links", ["Github"], icon_list=["github"], href_list=["https://github.com/taya-nicholas/basecampui"]),
        footer=DropdownFooter(),
        hx_boost="true", hx_target="#main-content", hx_select="#main-content", hx_ext="preload", preload="mouseover",
    )

In [ ]:
def TopBar(category="", component="", **kwargs):
    breadcrumb_items = []
    if category:  breadcrumb_items.append(category.title())
    if component: breadcrumb_items.append(slug_map.get(component, component))
    return Div(
        Div(
            ToggleButton(),
            SeparatorVertical(),
            Breadcrumb(breadcrumb_items or ["Index"]),
            cls="flex items-center gap-2"
        ),
        ThemeSwitcher(),
        cls="flex justify-between items-center w-full px-3 overflow-hidden py-1 border-b"
    )

In [ ]:
def sidebar_layout(fn):
    @wraps(fn)
    def inner_fn(*args, **kwargs):
        return Div(
            SiteSidebar(),
            Main(
                TopBar(**kwargs),
                Div(
                    fn(*args, **kwargs),
                    cls="flex flex-col flex-1 overflow-y-auto min-h-0",
                ),
                cls="h-full flex flex-col",
                id="main-content"
            ),
            cls="h-screen w-full",
        )
    return inner_fn

### Chat interface

In [ ]:
from lisette import *
import lancedb
from lancedb.table import LanceTable
from google import genai
from google.genai import types


In [ ]:
client = genai.Client()
db = lancedb.connect("./embeddings")

In [ ]:
def mk_emb_table(name:str, doc_list:list, recreate=False):
    if not recreate and name in db.table_names():
        print(f"Table({name}) already exists. Use recreate=True to recreate.")
        return db.open_table(name)
    results = client.models.embed_content(
        model="gemini-embedding-001", contents=doc_list,
        config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT"))
    embeds = [{"text": doc, "vector": vec.values} for vec, doc in zip(results.embeddings, doc_list)]
    db.drop_table(name, ignore_missing=True)
    return db.create_table(name, embeds)

def search(query:str, table:LanceTable, limit=5):
    query_emb = client.models.embed_content(
        model="gemini-embedding-001", contents=query,
        config=types.EmbedContentConfig(task_type="RETRIEVAL_QUERY")).embeddings[0].values
    return table.search(query_emb).select(["text", "_distance"]).limit(limit).to_list()

def merge_chunks(chunks, filename, min_size=600):
    merged, cur = [], ""
    for c in chunks:
        cur = f"{cur}\n\n{c}" if cur else c
        if len(cur) >= min_size: merged.append(f"File: {filename}\n\n{cur}"); cur = ""
    return merged + [f"File: {filename}\n\n{cur}"] if cur else merged

In [ ]:
# Build sources from documentation notebooks
sources = [f"File: {cat}\n\n" + "\n".join(o.source for o in comp)
           for cat, comp_dict in documentation.items() for comp in comp_dict.values()]

# Add chunked utils.py
sources.extend(merge_chunks(Path("../basecampui/utils.py").read_text().split('\n\n'), "Utils"))

# Create embedding table
docs_full = mk_emb_table("docs_full", sources, recreate=False)

Table(docs_full) already exists. Use recreate=True to recreate.


In [ ]:
sp = """
You exist in a documentation website and your goal is to answer questions about the library: basecampui.
BasecampUI is a ui library for FastHTML (written in python) using the css library basecoat (which is a vanilla css and js copy of shadcn).

When answering questions you will be provded with RAG context about the relevant code. Answer as if you were a regular person. An llm answers with lots of formatting, verbose descriptions, sections, em-dashes, etc. A regular person answers simply. They understand the core of what someone is asking, and just answer that part.

Note that users don't have access to the RAG context that you will be provided with.
Formatting: Outputs allow for headers and ```python code blocks but no other md outputs.
"""

chat = AsyncChat(model="groq/openai/gpt-oss-20b", sp=sp)

In [ ]:
@patch()
@delegates(AsyncChat.__call__, but=["msg"])
def query(self: Chat|AsyncChat, text, **kwargs):
    rag_context = "CONTEXT:" + "\n\n".join([
        o["text"] for o in
        search(text, docs_full, limit=3)
    ])
    query_text = f"{rag_context}\n USER QUERY: {text}"
    return self(query_text, **kwargs)

In [ ]:
def ChatMessage(content:str, msg_type="", cls="", rounding="", color="", txt_cls="px-4 py-3 whitespace-pre-wrap", id="", **kwargs): 
    return Div(
        Div(
            Div(
                P(msg_type, cls="ml-2"), cls=f"text-xs text-muted-background border-b border-border hover:cursor-pointer {rounding} rounded-b-none {color}"),
                Div(content, cls=f"{txt_cls}", id=f"{id}-ctn", **kwargs),
        ),
        cls=f"border border-border text-sm {cls} {rounding}",
        id=id,
    )

In [ ]:
def ChatAssistant(content:str, id="", **kwargs):
    return ChatMessage(content, msg_type="Assistant", cls="bg-background w-[90%] ml-auto mb-2", rounding="rounded-2xl", color="bg-rose-700/70", id=id, **kwargs)

In [ ]:
def ChatAssistantMd(md, id="", **kwargs):
    return ChatMessage(
        Div(*split_md(md).map(render_md_item)),
        msg_type="Assistant", 
        cls="bg-background w-[90%] ml-auto mb-2", 
        rounding="rounded-2xl", 
        color="bg-rose-700/70",
        txt_cls="px-4 py-3",
        id=id,
        **kwargs
    )

In [ ]:
chat = AsyncChat(model="anthropic/claude-haiku-4-5")
# chat = Chat(model="anthropic/claude-haiku-4-5")

@app.ws("/test_stream")
async def test_stream(msg, send, session):
    msg_list = session.get("msg_list", [])
    id = f"out-msg-{len(msg_list)}"
    id_ctn = f"{id}-ctn"
    await send(Div(ChatAssistant("", id=id), id="chat-interface-test", hx_swap_oob="beforeend"))
    
    msg = "Test response. Please give a python code block as well."
    stream = await chat(msg, stream=True)
    async for r in stream:
        delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
        if delta and delta.content:
            await send(Div(delta.content, id=id_ctn, hx_swap_oob="beforeend"))
            
    final_response = r.choices[0].message.content
    await send(ChatAssistantMd(final_response, id=id, hx_swap_oob="outerHTML"))
    await send(Script("hljs.highlightAll();", id="highlight-script", hx_swap_oob="true"))
    msg_list.append(msg)
    msg_list.append(final_response)
    await send(Script(f"fetch('/save_chat', {{method:'POST', headers:{{'Content-Type':'application/json'}}, body:JSON.stringify({json.dumps(msg_list)})}})", id="save-script", hx_swap_oob="true"))



In [ ]:
from starlette.requests import Request

@rt("/save_chat")
async def post(session, req: Request):
    msg_list = await req.json()
    session["msg_list"] = msg_list
    return "ok"

In [ ]:
def render_msgs(msg_list:list):
    results = []
    for i, m in enumerate(msg_list):
        if (i % 2) == 0:
            results.append(ChatPrompt(m))
        else:
            results.append(ChatAssistantMd(m))         
    return results


In [ ]:
@rt
def reset_chat(session):
    session["msg_list"] = []

In [ ]:
@rt("/test/test/test")
def get(session):
    msg_list = session.get("msg_list", [])
    return Div(
        Script(id="highlight-script"),
        Script(id="save-script"),
        Button("TEST4", hx_ext="ws", ws_connect="/test_stream", ws_send=True),
        Button("Clear", hx_post=reset_chat, hx_target="#chat-interface-test", hx_swap="innerHTML", cls=ButtonT.outline),
        ChatInterface(
            *render_msgs(msg_list),
            id="chat-interface-test",
        )
    )

In [ ]:
@app.ws("/send_msg_ws")
async def send_msg_ws(text:str, model_select:str, send, session):
    chat = AsyncChat(model=model_select, sp=sp)
    msg_list = session.get("msg_list", [])
    chat.h = mk_msgs(msg_list)
    id = f"out-msg-{len(msg_list)}"
    id_ctn = f"{id}-ctn"
    await send(Div(ChatPrompt(text), id="chat-interface", hx_swap_oob="beforeend"))
    await send(Div(ChatAssistant("", id=id), id="chat-interface", hx_swap_oob="beforeend"))

    stream = await chat.query(text, stream=True)
    async for r in stream:
        delta = r.choices[0].delta if hasattr(r.choices[0], 'delta') else None
        if delta and delta.content:
            await send(Div(delta.content, id=id_ctn, hx_swap_oob="beforeend"))
            
    final_response = r.choices[0].message.content
    await send(ChatAssistantMd(final_response, id=id, hx_swap_oob="outerHTML"))
    await send(Script("hljs.highlightAll();", id="highlight-script", hx_swap_oob="true"))
    msg_list.append(text)
    msg_list.append(final_response)
    await send(Script(f"fetch('/save_chat', {{method:'POST', headers:{{'Content-Type':'application/json'}}, body:JSON.stringify({json.dumps(msg_list)})}})", id="save-script", hx_swap_oob="true"))

In [ ]:
def ListBox(*contents, id, name=None, trigger_btn=ListboxTriggerButton(), search_bar=SearchBar(), side="bottom", align="left"):
    trigger_btn.attrs.update(dict(aria_controls=f"{id}-listbox", id=f"{id}-trigger"))
    if search_bar: search_bar.attrs.update(dict(aria_controls=f"{id}-listbox", aria_labelledby=f"{id}-trigger"))
    nm = name or id
    return Div(
        trigger_btn,
        Div(
            search_bar,
            Div(
                *contents,
                role="listbox", id=f"{id}-listbox", 
                aria_orientation="vertical", 
                aria_labelledby=f"{id}-trigger"
            ),
            id=f"{id}-popover", data_popover=True, aria_hidden="true", data_side=side, data_align=align,
        ),
        Input(type="hidden", name=nm, value=""),
        id=id, cls="select"
    )

In [ ]:
@delegates(ListBox, but=["search_bar"])
def Select(*args, searchable=False, **kwargs):
    search_bar = SearchBar() if searchable else None
    return ListBox(*args, search_bar=search_bar, **kwargs)

In [ ]:
def ChatInput(cls="", ta_cls="", width="w-96", btn_list=None, select_list=None, usage_int=None, **kwargs):
    select_btn = select_list and Select(
                *map(SelectItem, select_list),
                trigger_btn=ListboxTriggerButton(icon=None, cls=ButtonT.sm_ghost),
                id="model_select", name="model_select", side="top",
            )
    usage_div = usage_int and Div(f"{usage_int}% used", cls="text-muted-foreground text-sm")
    btn_list = btn_list or []
    return Form(
        Textarea(placeholder="Ask, Search or Chat...", cls=f"textarea pr-10 min-h-27 pb-12 {ta_cls} {width}", name="text"),
        Footer(
            *btn_list,
            select_btn,
            Div(
                usage_div,
                Button(Icon("arrow-up"), cls=ButtonT.icon_primary + "rounded-full size-6", **tooltip_kwargs("ctrl+enter"), name="submit-button"),
                cls="flex ml-auto gap-2 items-center",
            ),
            cls=f"absolute bottom-0 px-3 pb-3 pt-1.5 flex items-center {width} gap-2"
        ),
        cls=f"relative {cls}",
        **kwargs
    )

In [ ]:
@rt("/demos/documentation-rag/test")
@sidebar_layout
def documentation_rag(session, category:str="demos", component:str="Documentation-RAG"):
    msg_list = session.get("msg_list", [])
    return Div(
        Script(id="highlight-script"),
        Script(id="save-script"),
        ChatInterface(
            *render_msgs(msg_list),
            chat_input=ChatInput(
                select_list=["groq/openai/gpt-oss-20b", "groq/openai/gpt-oss-120b", "anthropic/claude-haiku-4-5"], 
                cls="mb-1 mt-0", ta_cls="rounded-t-none", width="w-full",
                hx_ext="ws", ws_connect="/send_msg_ws", ws_send=True,
                btn_list=[
                    Button(
                        Icon("trash-2"), type="button", cls=ButtonT.icon_ghost + "rounded-full size-6", **tooltip_kwargs("Clear chat"), hx_post=reset_chat, hx_target="#chat-interface", hx_swap="innerHTML",
                    )
                ]
            ),
            cls="flex-1",
            id="chat-interface",
        ),
        cls="flex flex-1",
    )

### Dyanmic routes

#### Display demo card

In [ ]:
def find_demos(nb_cell_list:list):
    return L(nb_cell_list).filter(Self.source.startswith("#| demo"))

In [ ]:
def exec_demo(cell:dict):
    ns = {}
    exec(cell.source, globals(), ns)
    name, fn = next(iter(ns.items())) 
    return (name, fn, cell.source)

In [ ]:
def render_demo_cards(nb_cell_list):
    return [DisplayDemoCard(*exec_demo(d)) for d in find_demos(nb_cell_list)]

In [ ]:
def DisplayDemoCard(nm, fn, src):
    return Div(
        Tabs(
            contents=[
                CardOutline(fn(), cls="rounded-lg min-h-96 flex justify-center items-center py-5"),
                Card(
                    Pre(
                        Code(src, cls="language-python-custom", style="background: transparent; overflow: visible;"),
                    ), 
                cls="rounded-lg min-h-96 flex text-sm overflow-auto overflow-y-auto",
                ),
            ],
            tablist=["preview", "code"],
            id=f"{nm}-tab",
            cls="w-full",
            nav_cls="w-fit ml-auto",
        ),
        cls="w-full",
    )

#### Source code accordion

In [ ]:
def strip_meta(text, prefixes=["#| exports", "#| export", "\n"]):
    for prefix in prefixes:
        text = text.removeprefix(prefix)
    return text

In [ ]:
def find_sourcecode(nb_cell_list):
    return L(nb_cell_list).filter(Self.source.startswith(("#| demo", "#| preview")), negate=True)[1:]

In [ ]:
def get_code_and_output(cell):
    code = strip_meta(cell["source"])
    output = [strip_meta(get_output_text(o)) for o in cell["outputs"]]
    return code, output

In [ ]:
def get_clean_output(output):
    if not isinstance(output, dict): return str(output)
    data = output.get("data", output)
    text = data.get("text/plain") or data.get("text/html") or data.get("text", "")
    return strip_meta(text)

In [ ]:
def render_cell(c):
    if c["cell_type"] == "markdown": return ChatNote(c["source"])
    code = strip_meta(c["source"])
    out = get_clean_output(c["outputs"][0]) if c["outputs"] else None
    return ChatCodeAndOutput(code, out)

In [ ]:
def AccordionSourceCode(nb_cell_list):
    contents = find_sourcecode(nb_cell_list).map(render_cell)
    return Accordion(
        AccordionItem(
            "Source code",
            ChatInterface(
                *contents,
                inner_cls="border rounded-lg",
            )
        ),
        cls="mt-4"
    )

#### Default component route

In [ ]:
@rt("/{category}/{component}")
@sidebar_layout
def get(category: str, component: str):
    nb_cell_list = documentation.get(category.title(), {}).get(component, {})
    if not nb_cell_list:
        return Div(H3("This page doesn't exist yet...", cls="text-xl mt-10"), cls="flex justify-center")
    return Div(
        H3(slug_map[component], cls="text-xl font-semibold tracking-tight mt-3"),
        Div(*render_demo_cards(nb_cell_list), cls="flex flex-col gap-3"),
        AccordionSourceCode(nb_cell_list),
        cls="flex flex-col w-full lg:max-w-4xl mx-auto",
    )

### Index

In [ ]:
def PopoverFastHTML():
    return Div(
        Img(src="https://www.fastht.ml/docs/logo.svg"),
        P("A Python framework for building modern web applications using pure Python.", cls="text-muted-foreground text-sm mt-2"),
        Div(
            A("Docs", href="https://www.fastht.ml/docs/", cls="text-primary underline text-sm"),
            A("GitHub", href="https://github.com/AnswerDotAI/fasthtml", cls="text-primary underline text-sm"),
            cls="flex gap-4 mt-3"
        ),
    )

In [ ]:
def PopoverBasecoat():
    return Div(
        H1("Basecoat", cls="text-2xl font-bold"),
        P("A collection of ShadCN styled UI components using lightweight CSS and vanilla JavaScript", cls="text-muted-foreground text-sm mt-2"),
        Div(
            A("Docs", href="https://basecoatui.com", cls="text-primary underline text-sm"),
            A("GitHub", href="https://github.com/hunvreus/basecoat", cls="text-primary underline text-sm"),
            cls="flex gap-4 mt-3"
        ),
    )

In [ ]:
def PopoverFastlucide():
    return Div(
        H1("fastlucide", cls="text-2xl font-bold"),
        P("A simple and efficient implementation of lucide icons in python. It also offers methods to transform and combine existing icons into new ones.", cls="text-muted-foreground text-sm mt-2"),
        Div(
            A("Docs", href="https://answerdotai.github.io/fastlucide/", cls="text-primary underline text-sm"),
            A("GitHub", href="https://github.com/AnswerDotAI/fastlucide", cls="text-primary underline text-sm"),
            cls="flex gap-4 mt-3"
        ),
    )

In [ ]:
def split_md(md):
    return L(re.split(r"(```.*?```|\n)", md, flags=re.DOTALL)).filter(Self.strip())

In [ ]:
def get_md_from_notebook(filename, as_list=True):
    index_cells = nbformat.read(filename, as_version=4)["cells"]
    md = "\n".join([c.source for c in index_cells if c.cell_type == "markdown"])
    if as_list:
        md = split_md(md)
    return md

In [ ]:
md_items = get_md_from_notebook("index.ipynb")
md_items

['# BasecampUI', "BasecampUI is a FastHTML UI library built on top of the excellent Basecoat. It's design goal is to create complex UI components with minimal boilerplate, while still allowing as much customization as possible", '## Quickstart\r', '```sh\r\npip install basecampui\r\n```', "```python\r\nfrom fasthtml.common import *\r\nfrom basecampui.all import *\r\n\r\napp = fastHTML()\r\nrt = app.route\r\n\r\n@rt\r\ndef index():\r\n    return Div(\r\n        Breadcrumb(['Home', 'Documents', 'Doc 1']),\r\n    )\r\n\r\nserve()\r\n```", '## Using Icons', 'BasecampUI uses fastlucide for icons. For development purposes, basecampui.all provides a standalone Icon function that creates a new SVG for each icon instance, making it easy to quickly iterate.', '```python\nDiv(\n    Icon("apple")\n)\n```', 'If you are happy with your set of icons, you can predefine them on app setup, and use the fastlucide spritesheet singleton instead of Icon function. This create a reference sprite sheet that al

In [ ]:
@rt
@sidebar_layout
def index():
    replace_map = {
        "FastHTML": TextPopover(PopoverFastHTML(), "FastHTML", "fhtml-popup", side="bottom"),
        "Basecoat": TextPopover(PopoverBasecoat(), "Basecoat", "bcoat-popup", side="bottom"),
        "fastlucide": TextPopover(PopoverFastlucide(), "fastlucide", "fastlucide-popup", side="bottom"),
    }
    return Div(
        *md_items.map(render_md_item, replace_map=replace_map),
        cls="flex flex-col w-full lg:max-w-4xl mx-auto"
    )
    